In [19]:
import json
import pickle
import pandas as pd

from data.tabular import TabularDataset

def get_original_test_labels(save_dir, data_path):
    # Load splits
    print(save_dir)
    splits_file = f"{save_dir}/splits_seed_1.json"
    with open(splits_file, 'r') as f:
        splits = json.load(f)
    
    # Load label encoder
    with open(f"{save_dir}/label_encoder.pkl", 'rb') as f:
        encoder = pickle.load(f)
    
    # Recreate dataset to access original indices
    dataset = TabularDataset(data_path, save_dir)
    
    # Get all test indices (ID + OOD)
    all_test_indices = splits['test_idx']
    
    # Get encoded labels and convert to original
    encoded_labels = dataset.targets[all_test_indices].cpu().numpy()
    original_labels = encoder.inverse_transform(encoded_labels)
    
    # Get original texts from raw data
    with open(data_path) as f:
        data_json = json.load(f)
    
    text_df = pd.DataFrame([
        {'text': row['text'], 'label': row['mappings'][0]['attack_id']}
        for row in data_json['sentences']
        if len(row['mappings']) > 0
    ])
    
    return pd.DataFrame({
        'text': text_df.iloc[all_test_indices]['text'].values,
        'original_label': [label.split('.')[0] for label in original_labels]
    })

# Usage
test_df = get_original_test_labels(
    save_dir="saves",
    data_path="data/tabular/training-data.json"  # Replace with actual path
)
print(test_df.head())

test_df.to_csv("saves/test_labels.csv", index=False)


saves
Number of categories: 50


/home/pks230000/miniconda3/envs/torch_p313/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


                                                text original_label
0          has the ability to communicate over HTTP.          T1071
1  In addition to obfuscation techniques, it also...          T1518
2  can use `cmd.exe` to execute commands on a vic...          T1059
3  attempted to masquerade their files as popular...          T1036
4                      to start the download process          T1105
